## \[Data Science\] Full stack training

- campo chave para junção das tabelas: `grower_document`
- tabela base: `md_grower_report`
  - corresponde ao cadastro de agricultores da Bayer

In [1]:
import pandas as pd
from math import isnan
from pathlib import Path

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
INPUT_FOLDER = Path.cwd().parent / "inputs"

In [4]:
csv_files = Path(INPUT_FOLDER).glob('**/*.csv')

In [5]:
df = {f"{csv_file.name[10:-4]}": pd.read_csv(csv_file) for csv_file in csv_files}

/var/folders/g0/342sbsdd56173k7xyxgl_jh1w0chb5/T/ipykernel_65002/670898405.py:1: DtypeWarning: Columns (38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = {f"{csv_file.name[10:-4]}": pd.read_csv(csv_file) for csv_file in csv_files}
/var/folders/g0/342sbsdd56173k7xyxgl_jh1w0chb5/T/ipykernel_65002/670898405.py:1: DtypeWarning: Columns (1,2,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = {f"{csv_file.name[10:-4]}": pd.read_csv(csv_file) for csv_file in csv_files}
/var/folders/g0/342sbsdd56173k7xyxgl_jh1w0chb5/T/ipykernel_65002/670898405.py:1: DtypeWarning: Columns (37,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = {f"{csv_file.name[10:-4]}": pd.read_csv(csv_file) for csv_file in csv_files}


In [6]:
df.keys()

dict_keys(['sales', 'disque_intacta', 'md_grower_report', 'saved_seeds', 'entered_area'])

## Tables:

- **sales** => (871.264, 79) - (a_seller_document, a_buyer_document)
- **disque_intacta** => (385.716, 51)
- **md_grower_report** => (514.182, 38) - (a_grower_document)
- **saved_seeds** => (19.648, 72)
- **entered_area** => (51.262, 40)

## Dealing with duplicated data

An initial `left outer join` between `sales` and `md_grower_report` was resulting an amount greater than the number of rows of `sales`

This raised a red flag because probably there were duplicated keys in `md_grower_report`

Then, I started investigating

In [7]:
df_dup_detected = pd.merge(
    df["sales"],
    df["md_grower_report"],
    how="left",
    # on=None,
    left_on=["a_seller_document"],
    right_on=["a_grower_document"],
    # left_index=False,
    # right_index=False,
    # sort=True,
    suffixes=("_seller", "_grower"),
)
df_dup_detected.shape

(896150, 117)

`896.150` **>** `871.264` :: `24.886` extra amount (**!!!**)

In [8]:
grower_docs = df["md_grower_report"]["a_grower_document"]
grower_docs_unq = df["md_grower_report"]["a_grower_document"].unique()
print(f"Comparing grower_docs.size with grower_docs_unq.size => {len(grower_docs)} vs {len(grower_docs_unq)}")
assert len(df["md_grower_report"]["a_grower_document"]) == len(df["md_grower_report"]["a_grower_document"].unique())

Comparing grower_docs.size with grower_docs_unq.size => 514182 vs 461935


AssertionError: 

In [9]:
df["md_grower_report"].dropna(subset=["a_grower_document"]).shape

(475945, 38)

In [10]:
# droping empty keys
df["md_grower_report"] = df["md_grower_report"].dropna(subset=["a_grower_document"])

In [11]:
ids = df["md_grower_report"]["a_grower_document"]
df_grower_report_dup = df["md_grower_report"][ids.isin(ids[ids.duplicated()])].sort_values("a_grower_document")
df_grower_report_dup.shape

(28011, 38)

In [12]:
# droping duplicated keys
df["md_grower_report"] = df["md_grower_report"].drop_duplicates(subset=["a_grower_document"], keep='first')
df["md_grower_report"].shape

(461934, 38)

In [13]:
df_cleaned = pd.merge(
    df["sales"],
    df["md_grower_report"],
    how="left",
    # on=None,
    left_on=["a_seller_document"],
    right_on=["a_grower_document"],
    # left_index=False,
    # right_index=False,
    # sort=True,
    suffixes=("_seller", "_grower"),
)
df_cleaned.shape

(871264, 117)

`871.264` **=** `871.264` :: **ZERO** extra amount (*first mission accomplished*)

In [14]:
df_final = pd.merge(
    df_cleaned,
    df["md_grower_report"],
    how="left",
    # on=None,
    left_on=["a_buyer_document"],
    right_on=["a_grower_document"],
    # left_index=False,
    # right_index=False,
    # sort=True,
    suffixes=("_buyer", "_grower_2"),
)
df_final.shape

(871264, 155)

`871.264` **=** `871.264` :: **ZERO** extra amount (*double check done*)

notice that the number of column are equal to `79` **+** `38` **+** `38` (**=** `155`)

number of columns in `sales` = 79 / number of columns in `md_grower_report` = 38

## Analyzing columns

It would be interesting to have some metrics related to each column of each table available

These metrics would help to filter candidates to be used as features in our clustering model

Some suggested metrics:
- number_of_values (`done`)
- entropy
- uniqueness_rate (`done`)
- count_unique_values (`done`)
- numerical_values_rate (`done`)
- alphabetical_values_rate (`done`)
- mean_numerical_values
- mean_alphabetical_values
- std_numerical_chars_in_values
- std_alphabetical_chars_in_values
- mean_special_chars_in_values
- mean_words_in_values
- std_special_chars_in_values
- std_words_in_values
- percent_none_values (`done`)
- count_none_values (`done`)
- min_value
- max_value
- median_value
- mode_value (`done`)

In [15]:
def number_of_values(series):
    return len(series)

In [16]:
def uniqueness_rate(series):
    return len(series.unique()) / len(series)

In [17]:
def count_unique_values(series):
    return len(series.unique())

In [18]:
def numerical_values_rate(series):
    return sum([1 if str(val).isnumeric() else 0 for val in series]) / len(series)

In [19]:
def percent_none_values(series):
    sum = 0
    for val in series:
        try:
            if not str(val):  # checking falsy values
                sum += 1
            elif str(val).isalpha() and isnan(float(val)):  # checking nan values
                sum += 1
        except ValueError:
            continue
    return sum / len(series)

In [20]:
def count_none_values(series):
    sum = 0
    for val in series:
        try:
            if not str(val):  # checking falsy values
                sum += 1
            elif str(val).isalpha() and isnan(float(val)):  # checking nan values
                sum += 1
        except ValueError:
            continue
    return sum

In [21]:
def mode_value(series):
    aux = {}
    for val in series:
        aux[val] = aux.get(val, 0) + 1
    max_key, max_val = (0, 0)
    for k, v in aux.items():
        if v > max_val:
            max_key = k
            max_val = v
    return max_key, max_val

In [22]:
df["md_grower_report"].columns

Index(['Unnamed: 0', 'id', 'grower_document_type', 'a_grower_document',
       'a_grower_name', 'a_grower_fantasy_name', 'a_email', 'collaborator',
       'cooperative', 'groundbreaker', 'precertificate', 'precertificate_from',
       'precertificate_to', 'a_address', 'address_type', 'additional_info',
       'neighborhood', 'state_id', 'state', 'city', 'city_id', 'region',
       'district', 'agreement_id', 'agreement_template_id', 'technology',
       'agreement_type', 'status', 'bid', 'creation_user', 'creation_date',
       'sent_user', 'sent_date', 'approval_user', 'approval_date', 'comments',
       'region_grower', 'district_grower'],
      dtype='object')

In [23]:
metrics = [
    "number_of_values",
    "uniqueness_rate",
    "count_unique_values",
    "numerical_values_rate",
    "percent_none_values",
    "count_none_values",
    "mode_value",
]

In [24]:
def analysis_per_column(dff, column):
    return {metric: globals()[metric](dff[column]) for metric in metrics}

In [25]:
analysis_per_column(df["md_grower_report"], "a_grower_document")

{'number_of_values': 461934,
 'uniqueness_rate': 1.0,
 'count_unique_values': 461934,
 'numerical_values_rate': 0.0,
 'percent_none_values': 0.0,
 'count_none_values': 0,
 'mode_value': (551155.0, 1)}